# Exploring eBay Car Sales Data

## Introduction

This project aims to clean and analyse a dataset of used cars from *eBay Kleinanzeigen*, a classifieds section of the German eBay website.

The dataset that will be used for this project is a modified version of the one found [here](https://data.world/data-society/used-cars-data). The data was modified in the following ways:
* Only 50,000 data points were sampled from the full dataset.
* The dataset was "dirtied" to more closely resemble what would be expected from a pre-cleaned, scraped dataset.


In [87]:
import numpy as np
import pandas as pd

In [88]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The summary of our dataset shows that there are twenty columns, five of which contain null values (` vehicleType`, `gearbox`, `model`, `fuelType`, `notRepairedDamage`). There are two dtypes: object and int64, and, lastly, there are columns that that contain dates stored as strings.

As the column names are in camelcase, as opposed to the preferred snakecase (for python), let's begin by cleaning the column names to make the data easier to work with:

## Clean Columns

In [89]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Changes to be made:
* Change the columns from camelcase to snakecase.
* Rewording some column names to more more accurate descriptions.

In [90]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Initial Data Exploration and Cleaning

Exploring the data to find any obvious ares where the data can be cleaned:

In [91]:
autos.describe(include="all")    # 'include=all' to get both categorical and numeric columns

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-10 15:36:24,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Initial observations:

There are a number of text columns where all (or nearly all) of the values are the same:

* `seller`
* `offer_type`

The `num_photos` column doesn't seem to contain any values:

In [92]:
autos["num_photos"].value_counts()

0    50000
Name: num_photos, dtype: int64

It's confirmed that the `num_photos` column only contains 0 in every entry. Therefore, this can be removed along with the `seller` and `offer_type` columns:

In [93]:
autos.drop(["seller", "offer_type", "num_photos"], axis=1, inplace=True)

Now to move onto columns that have numerical values stored as strings. For example, the `price` and `odometer` columns. 

For further analysis, we will need to:

* Remove any non-numerical characters.
* Convert the columns to a numeric dtype.
* Rename the columns to make them more clear (e.g. changing odometer to odometer_km after removing the 'km')

Starting with price:

In [94]:
autos["price"].head(3)    # we can see that after removing the "$" and "," characters we will be left with numeric values.

0    $5,000
1    $8,500
2    $8,990
Name: price, dtype: object

Removing unwanted characters and converting to a float:

In [95]:
autos["price"] = (autos["price"]
                        .str.replace(",","")  
                        .str.replace("$","")
                        .astype(float))

autos["price"].head(3)

0    5000.0
1    8500.0
2    8990.0
Name: price, dtype: float64

Now, to do the same with odometer:

In [96]:
autos["odometer"].head(3)   

0    150,000km
1    150,000km
2     70,000km
Name: odometer, dtype: object

In [97]:
autos["odometer"] = (autos["odometer"]     # need to remove the comma (,) and 'km'
                            .str.replace("km","")
                            .str.replace(",","")
                            .astype(float))

autos["odometer"].head(3)

0    150000.0
1    150000.0
2     70000.0
Name: odometer, dtype: float64

Renaming columns:

In [98]:
autos.rename({"price":"price_$", "odometer":"odometer_km"}, axis=1, inplace=True)

In [99]:
autos.head(2)

,date_crawled,name,price_$,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000.0,control,bus,2004,manuell,158,andere,150000.0,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500.0,control,limousine,1997,automatik,286,7er,150000.0,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08


## Exploring Odometer and Price

Looking for outliers for the 'odometer_km' and 'price_$' columns:

In [100]:
autos["price_$"].unique().shape   # to see how many unique values

(2357,)

In [101]:
autos["price_$"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_$, dtype: float64

A description of the price column shows that there are listings at $0 as a minimum and 1.000000e+08 as a maximum, which seems very high.

Having a closer look:

In [102]:
autos["price_$"].value_counts().sort_index(ascending=False).head(15)

99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
350000.0      1
345000.0      1
299000.0      1
295000.0      1
265000.0      1
Name: price_$, dtype: int64

We can see that after the 350,000 price mark, the prices seems to jump dramatically, from 999,990 all the way up to 99,999,999. As eBay is a auction website, it's possible that vehicles were listed at these prices. However, anything above 350,000 seems unrealistic, especially given the dramatic jump. 

In [103]:
autos["price_$"].value_counts().sort_index().head(20)

0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
13.0       2
14.0       1
15.0       2
17.0       3
18.0       1
20.0       4
25.0       5
29.0       1
30.0       7
35.0       1
Name: price_$, dtype: int64

We can also see that there are a number of listings below the 35 dollar price. Again, as eBay is an auction site, this is entirely possible. Therefore, any listings above a 1 dollar opening will be kept, while those listed at 0 will be removed. In addition to removing listings at 0 dollars, I will also remove those over 350,000, as mentioned above.

In [104]:
## Removing the outliers ##

autos = autos[autos["price_$"].between(1, 350001)]

So the remaining vehicles we have in our dataset are those with the price listings between 1-350,000 dollars.

Now to check the odometer_km column:

In [105]:
autos["odometer_km"].unique().shape

(13,)

In [106]:
autos["odometer_km"].describe()

count     48565.000000
mean     125770.101925
std       39788.636804
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [107]:
autos["odometer_km"].value_counts()

150000.0    31414
125000.0     5057
100000.0     2115
90000.0      1734
80000.0      1415
70000.0      1217
60000.0      1155
50000.0      1012
5000.0        836
40000.0       815
30000.0       780
20000.0       762
10000.0       253
Name: odometer_km, dtype: int64

There doesn't appear to be anything out of the ordinary in the 'odometer_km' column. Thefore, this column will remain as it is.

## Exploring the Date Columns

There are five columns that should represent date values. Some were created by the crawler, and some came from the website itself.

As it stands, the `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values by pandas (displayed as the 'object' dtype). To be able to understand these dates quantitatively, we need to convert them into their numerical representations.

First we need to understand how these string columns are formatted:

In [108]:
autos[["date_crawled", "ad_created", "last_seen"]][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In each column, we can see that the first 10 characters represent the date (e.g. 2016-03-26), so to understand the date range distribution, we can extract just the date values:

For `date_crawled`:

In [109]:
(autos["date_crawled"]
             .str[:10]
             .value_counts(normalize=True, dropna=False)   # 'normalize=True' gives us the relative frequencies of the unique values.
             .sort_values())            

2016-04-07    0.001400
2016-04-06    0.003171
2016-03-18    0.012911
2016-04-05    0.013096
2016-03-06    0.014043
2016-03-13    0.015670
2016-03-05    0.025327
2016-03-24    0.029342
2016-03-16    0.029610
2016-03-27    0.031092
2016-03-25    0.031607
2016-03-17    0.031628
2016-03-31    0.031834
2016-03-10    0.032184
2016-03-26    0.032204
2016-03-23    0.032225
2016-03-11    0.032575
2016-03-22    0.032987
2016-03-09    0.033090
2016-03-08    0.033296
2016-03-30    0.033687
2016-04-01    0.033687
2016-03-29    0.034099
2016-03-15    0.034284
2016-03-19    0.034778
2016-03-28    0.034860
2016-04-02    0.035478
2016-03-07    0.036014
2016-04-04    0.036487
2016-03-14    0.036549
2016-03-12    0.036920
2016-03-21    0.037373
2016-03-20    0.037887
2016-04-03    0.038608
Name: date_crawled, dtype: float64

We can see that listings were crawled daily over a one month period (March-April, 2016). The distribution of the amount of listings crawled each day within this period is roughly uniform.

For `last_seen`:

In [110]:
(autos["last_seen"]
             .str[:10]
             .value_counts(normalize=True, dropna=False)
             .sort_values())            

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-18    0.007351
2016-03-08    0.007413
2016-03-13    0.008895
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-14    0.012602
2016-03-27    0.015649
2016-03-19    0.015834
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-26    0.016802
2016-03-23    0.018532
2016-03-25    0.019211
2016-03-24    0.019767
2016-03-21    0.020632
2016-03-20    0.020653
2016-03-28    0.020859
2016-03-22    0.021373
2016-03-29    0.022341
2016-04-01    0.022794
2016-03-12    0.023783
2016-03-31    0.023783
2016-04-04    0.024483
2016-03-30    0.024771
2016-04-02    0.024915
2016-04-03    0.025203
2016-03-17    0.028086
2016-04-05    0.124761
2016-04-07    0.131947
2016-04-06    0.221806
Name: last_seen, dtype: float64

The date the crawler last saw a listing allows us to determine on what day a listing was removed, presumably because the vehicle was sold.

In general, it's observed that as the date increases, the higher the frequency of 'last seen' values we see. This makes sense as the longer a listing is online, the more chance it has to be sold. 

However, during the last three days there is a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from previous days, it's more likely that these values are to do with the crawling period ending rather than indicating a dramatic spike in car sales.

For `ad_created`:

In [111]:
(autos["ad_created"]
             .str[:10]
             .value_counts(normalize=True, dropna=False)
             .sort_values())            

2016-01-14    0.000021
2016-02-16    0.000021
2016-02-07    0.000021
2016-02-22    0.000021
2015-06-11    0.000021
                ...   
2016-03-12    0.036755
2016-04-04    0.036858
2016-03-21    0.037579
2016-03-20    0.037949
2016-04-03    0.038855
Name: ad_created, Length: 76, dtype: float64

There is quite a large range of dates from when the ads were created. However, the distribution shows that most ads were created within the month that the respective listings were crawled. The oldest appears to be around 9 months.

To understand the distribution of `registration_year`:

In [112]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The registration year of a vehicle is very likely the year the vehicle was produced i.e. it represents its age. Here we can see that the min registration year is 1000, and the max is 9999. Obviously, we  need to investigate this.

## Dealing with Incorrect Registration Year Data

As the year 1000 is much earlier than the invention of cars, and that the year 9999 is in the future, there is incorrect data in our dataset. 

As this data were collected in 2016, we know that any registration year after this year is incorrect. Determining the earliest valid year is more difficult, however. Realistically, it could be any time after the 1900s.

Therefore, one option is to remove any listings that fall outside the registration years 1900-2016. However, before doing this, we want to determine what percentage of our data has invalid values in this column:

In [113]:
(~autos["registration_year"].between(1900,2016)).sum() / autos.shape[0]  # '~' is the pandas equivalent logic of 'IS NOT'

0.038793369710697

As less than 4% of our dataset contains invalid values, we will remove these rows:

In [114]:
autos = autos[autos["registration_year"].between(1900,2016)]

autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
Name: registration_year, dtype: float64

It appears that most vehicles listed were first registered in the last twenty years.

## Exploring Price by Brand